In [1]:
import yaml
import os
from misc import *
from bcg_colors import *
from pandas.api.types import is_bool_dtype
_ = setup_plotting(style='ticks', context='talk')

In [2]:
DATA_DIR = os.path.join(DATA, 'DE')
RESULTS_DIR = 'results_bootstrap'
DATA_DIR, RESULTS_DIR

('../data/DE', 'results_bootstrap')

In [3]:
LOLA_DB_DIR = 'resources/LOLA'
LOLA_DB = ['LOLAbcg/hg38']
LOLA_COLLECTIONS = [HOCOMOCO, EPI_ROADMAP, 'leukotypes']

In [4]:
FULL_BCG_PTHW_SET = [BCG_PATHWAYS] + ['{}_min15_max500'.format(lib) for lib in [KEGG, GO_BIO_PROCESS]]
WHICH_REGIONS = [GENE_AND_DISTAL_10kb, TSS_PROXIMAL]
RANK_METRIC = 'p.value'
TOP_N = 1000

#### Variance partitioning

In [5]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
print(annot_fn)
_annot_df = get_sample_annot(annot_fn)

for batch_corrected, normal_data, model, columns, random_effects, counts in [
    (True, False, 'varPartTop.batch_TSS_enr_blood_corrected.donor',
     ['SAMPLE.DONOR'], '(1|SAMPLE.DONOR)',
     os.path.join(DATA_DIR, 'blood_TSS_enr_corrected_combat_normalized_log2_CPM_PBMC.csv.gz')),
]:
    config = {}
    config['results_dir'] = RESULTS_DIR
    config['debug'] = False
    config['batch_corrected'] = batch_corrected
    config['dream'] = False
    config['var_part'] = True

    config['celltype'] = celltype
    config['model'] = model
    config['annot_fn'] = annot_fn
    config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
    config['counts_fn'] = counts if counts else os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))
    if normal_data:
        config['normal_data'] = normal_data
    
    _covariates = ' + '.join([c for c in columns if not random_effects or c not in random_effects.replace('(', '').replace(')', '').replace('1', '').replace('|', '').replace(' ', '').split('+')])
    config['design'] = '~ {covariates}{random_effects}'.format(
        covariates=_covariates,
        random_effects='{}{}'.format(' + ' if len(_covariates) != 0 else '', random_effects) if random_effects else ''
    )
    config['columns'] = columns + (['SAMPLE.VISIT'] if 'SAMPLE.VISIT' not in columns else [])
    
    config['splines'] = None 
    config['splines_df'] = None

    config['block_on_donor'] = False
    config['remove_wrong_batch'] = True
    config['remove_exclusions'] = True
    config['remove_300BCG315'] = False
    config['remove_evening'] = False
    config['complete_design'] = False
    config['useful_samples'] = False
    config['save_non_log_CPM'] = False
    config['do_not_correct'] = None

    config['volcano_fdr'] = None
    
    config['effect_size_filter'] = None
    config['rank_metric'] = RANK_METRIC
    config['top_n'] = TOP_N

    config['lola_db_dir'] = LOLA_DB_DIR
    config['lola_db'] = LOLA_DB
    config['lola_collections'] = LOLA_COLLECTIONS
    config['lola_remove_replicates'] = True
    config['lola_metric'] = 'pValueLog'
    config['lola_show_n'] = 20
    config['lola_fdr'] = 0.05

    _pathways = FULL_BCG_PTHW_SET
    _regions = WHICH_REGIONS
    config['enrichr_gene_sets'] = _pathways
    config['enrichr_regions'] = _regions
    config['enrichr_metric'] = 'P-value'
    config['enrichr_show_n'] = 20
    config['enrichr_fdr'] = 0.1
    config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries')

    de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
    for d in ['LOLA', 'Enrichr']:
        make_dir(de_dir, d)
    with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    print(config['counts_fn'])
    print(config['model'])
    print(config['design'])
    print('')

../metadata/complete_metadata.corrected.csv
../data/DE/blood_TSS_enr_corrected_combat_normalized_log2_CPM_PBMC.csv.gz
varPartTop.batch_TSS_enr_blood_corrected.donor
~ (1|SAMPLE.DONOR)



#### V1 factors - one full model

In [6]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
print(annot_fn)
_annot_df = get_sample_annot(annot_fn)
FACTORS = ['DONOR:AGE', 'DONOR:SEX', 'SAMPLE:VISIT_TIME_REAL', 'DONOR:BMI', 'DONOR:oralContraceptivesIncludingMen', 'SAMPLE:alcoholInLast24h']

config = {}
config['results_dir'] = RESULTS_DIR
config['visits'] = ['V1']
config['debug'] = False
config['batch_corrected'] = False
config['dream'] = False
config['celltype'] = celltype
config['model'] = '{approach}{batch}.FACTORS{blood}.TSS_enr.visit_time'.format(
    approach='V1',
    batch='.batch' if not config['batch_corrected'] else '.batch_corrected',
    blood='.blood' if config['celltype'] == 'PBMC' else ''
)

config['annot_fn'] = annot_fn
config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
if config['batch_corrected']:
    config['counts_fn'] = os.path.join(DATA_DIR, 'batch_corrected_normalized_log2_CPM_{celltype}.csv.gz'.format(celltype=config['celltype']))
else:
    config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))

_blood = safe_R_name(BLOOD)
_contrasts = [['DONOR.SEXM', 'DONOR.AGE', 'SAMPLE.VISIT_TIME_REAL', 'DONOR.BMI', 'DONOR.oralContraceptivesIncludingMenTrue', 'SAMPLE.alcoholInLast24hTrue']]
_enr_contrasts = _contrasts

config['columns'] = sorted(set(['DONOR.SEX', 'DONOR.AGE', 'SAMPLE.VISIT_TIME_REAL', 'DONOR.BMI', 'DONOR.oralContraceptivesIncludingMen', 'SAMPLE.alcoholInLast24h', 'RUN.TSS_ENRICHMENT'] + \
(['LAB.BATCH'] if not config['batch_corrected'] else []) + \
(_blood if config['celltype'] == 'PBMC' else [])))

config['design'] = '~ {covariates}'.format(covariates=' + '.join(config['columns']))

config['contrasts'] = _contrasts
config['enr_contrasts'] = _enr_contrasts
assert is_iterable(config['contrasts'][0])
assert is_iterable(config['enr_contrasts'][0])

# e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
config['splines'] = None 
config['splines_df'] = None

config['block_on_donor'] = False
config['remove_wrong_batch'] = True
config['remove_exclusions'] = True
config['remove_300BCG315'] = False
config['remove_evening'] = False
config['complete_design'] = False
config['useful_samples'] = False
config['save_non_log_CPM'] = False
config['do_not_correct'] = None

config['volcano_fdr'] = [0.05, 0.1, 0.2]

config['effect_size_filter'] = None
config['rank_metric'] = RANK_METRIC
config['top_n'] = TOP_N

config['lola_db_dir'] = LOLA_DB_DIR
config['lola_db'] = LOLA_DB
config['lola_collections'] = LOLA_COLLECTIONS
config['lola_remove_replicates'] = True
config['lola_metric'] = 'pValueLog'
config['lola_show_n'] = 20
config['lola_fdr'] = 0.05

_pathways = FULL_BCG_PTHW_SET
_regions = WHICH_REGIONS
config['enrichr_gene_sets'] = _pathways
config['enrichr_regions'] = _regions
config['enrichr_metric'] = 'P-value'
config['enrichr_show_n'] = 20
config['enrichr_fdr'] = 0.1
config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
for d in ['LOLA', 'Enrichr']:
    make_dir(de_dir, d)
with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)
print(config['model'])

../metadata/complete_metadata.corrected.csv
V1.batch.FACTORS.blood.TSS_enr.visit_time


#### V1 cytokines and inflammatory markers

In [7]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
_annot_df = get_sample_annot(fn=annot_fn)
print(annot_fn)

for cyto in _annot_df.columns[_annot_df.columns.str.contains('^CYTO:.*_good$|^CM:')]:
    config = {}
    config['results_dir'] = RESULTS_DIR
    config['visits'] = ['V1']
    config['debug'] = False
    config['batch_corrected'] = False
    config['dream'] = False
    config['celltype'] = celltype
    config['model'] = 'V1{batch}{covar}{blood}.TSS_enr.visit_time{cyto}'.format(
        batch='.batch' if not config['batch_corrected'] else '.batch_corrected',
        covar='.sex.age{}'.format('.bmi.oralContra' if 'CM:' in cyto else ''),
        blood='.blood' if config['celltype'] == 'PBMC' else '',
        cyto='.{}'.format(safe_R_name(cyto))
    )
        
    config['annot_fn'] = annot_fn
    config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
    if config['batch_corrected']:
        config['counts_fn'] = os.path.join(DATA_DIR, 'batch_corrected_normalized_log2_CPM_{celltype}.csv.gz'.format(celltype=config['celltype']))
    else:
        config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))
    
    _contrasts = [[safe_R_name(cyto)]]
    _enr_contrasts = _contrasts
    _blood = safe_R_name(BLOOD)

    config['columns'] = sorted(set([safe_R_name(cyto), 'DONOR.SEX', 'DONOR.AGE', 'RUN.TSS_ENRICHMENT', 'SAMPLE.VISIT_TIME_REAL'] + \
    (['LAB.BATCH'] if not config['batch_corrected'] else []) + \
    (_blood if config['celltype'] == 'PBMC' else []) + \
    (['DONOR.BMI', 'DONOR.oralContraceptivesIncludingMen'] if 'CM:' in cyto else [])))

    config['design'] = '~ {covariates}'.format(covariates=' + '.join(config['columns']))

    config['contrasts'] = _contrasts
    config['enr_contrasts'] = _enr_contrasts
    assert is_iterable(config['contrasts'][0])
    assert is_iterable(config['enr_contrasts'][0])

    # e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
    config['splines'] = None 
    config['splines_df'] = None

    config['block_on_donor'] = False
    config['remove_wrong_batch'] = True
    config['remove_exclusions'] = True
    config['remove_300BCG315'] = False
    config['remove_evening'] = False
    config['complete_design'] = False
    config['useful_samples'] = False
    config['save_non_log_CPM'] = False
    config['do_not_correct'] = [safe_R_name(cyto)]

    config['volcano_fdr'] = [0.05, 0.1, 0.2]

    config['effect_size_filter'] = None
    config['rank_metric'] = RANK_METRIC
    config['top_n'] = TOP_N

    config['lola_db_dir'] = LOLA_DB_DIR
    config['lola_db'] = LOLA_DB
    config['lola_collections'] = LOLA_COLLECTIONS
    config['lola_remove_replicates'] = True
    config['lola_metric'] = 'pValueLog'
    config['lola_show_n'] = 20
    config['lola_fdr'] = 0.05

    _pathways = FULL_BCG_PTHW_SET
    _regions = WHICH_REGIONS
    config['enrichr_gene_sets'] = _pathways
    config['enrichr_regions'] = _regions
    config['enrichr_metric'] = 'P-value'
    config['enrichr_show_n'] = 20
    config['enrichr_fdr'] = 0.1
    config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

    de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
    for d in ['LOLA', 'Enrichr']:
        make_dir(de_dir, d)
    with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    print(config['model'])

../metadata/complete_metadata.corrected.csv
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.ADA_P00813
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.AXIN1_O15169
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.Beta.NGF_P01138
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CASP.8_Q14790
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL11_P51671
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL19_Q99731
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL20_P78556
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL23_P55773
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL25_O15444
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL28_Q9NRJ3
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL3_P10147
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CCL4_P13236
V1.batch.sex.age.bmi.oralContra.blood.TSS_enr.visit_time.CM.CD244_Q9BZW8
V1.batc

#### Seasonal changes in ATAC-seq

In [8]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
print(annot_fn)
_annot_df = get_sample_annot(annot_fn)
COEFS = pd.read_csv(os.path.join(METADATA, 'selected_factors.csv'), comment='#', header=None)[0]
COEFS = COEFS[COEFS.str.contains('^DONOR:')].tolist()

for factor in ['season'] + COEFS:
        config = {}
        config['results_dir'] = RESULTS_DIR
        config['debug'] = False
        config['batch_corrected'] = factor == 'season'
        config['dream'] = False
        config['celltype'] = celltype
        config['model'] = 'fixed.{approach}{batch}.sex.age{blood}.TSS_enr.visit_time{factor}_interaction'.format(
            approach='donor_as_mixed',
            batch='.batch' if not config['batch_corrected'] else '.batch_corrected',
            blood='.blood' if config['celltype'] == 'PBMC' else '',
            factor='.{}'.format(safe_R_name(factor)) if factor else ''
        )

        config['annot_fn'] = annot_fn
        config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
        if config['batch_corrected']:
            config['counts_fn'] = os.path.join(DATA_DIR, 'batch_corrected_normalized_log2_CPM_{celltype}.csv.gz'.format(celltype=config['celltype']))
        else:
            config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))

        if factor == 'season':
            # don't ask me
            _contrasts = [['DONOR.IC_DATE_2PI_SIN.{}'.format(visit),
                           '{}.DONOR.IC_DATE_2PI_COS'.format(visit)] for visit in ['V2', 'V3']] + \
                         [['V2', 'V3'],
                          ['SAMPLE.VISIT_DATE_2PI_SIN', 'SAMPLE.VISIT_DATE_2PI_COS']]
        else:
            _contrasts = [['{}{}.{}'.format(
                safe_R_name(factor),
                'True' if is_bool_dtype(_annot_df[factor].dtype) else 'M' if factor == 'DONOR:SEX' else '',
                visit
            ) for visit in ['V2', 'V3']]]
        _enr_contrasts = _contrasts
        
        _blood = safe_R_name(BLOOD)
        config['columns'] = sorted(set(['SAMPLE.DONOR', 'SAMPLE.VISIT', 'SAMPLE.VISIT_TIME_REAL', 'DONOR.IC_TIME_REAL', 'DONOR.SEX', 'DONOR.AGE', 'RUN.TSS_ENRICHMENT'] + \
        (['LAB.BATCH'] if not config['batch_corrected'] else []) + \
        (_blood if config['celltype'] == 'PBMC' else []) + \
        (['DONOR.IC_DATE_2PI_SIN', 'DONOR.IC_DATE_2PI_COS', 'SAMPLE.VISIT_DATE_2PI_SIN', 'SAMPLE.VISIT_DATE_2PI_COS'] if factor == 'season' else \
         [safe_R_name(factor), safe_R_name(factor.replace('DONOR:IC_TIME', 'SAMPLE:VISIT_TIME').replace('DONOR:IC_', 'SAMPLE:'))] \
         if factor and factor.startswith('DONOR:IC_') else [safe_R_name(factor)] if factor else []
        )))

        if factor == 'season':
            config['design'] = '~ DONOR.IC_DATE_2PI_SIN * SAMPLE.VISIT + DONOR.IC_DATE_2PI_COS * SAMPLE.VISIT + {covariates}'.format(
                covariates=' + '.join([c for c in config['columns'] if c not in ['SAMPLE.DONOR', 'DONOR.IC_TIME_REAL', 'SAMPLE.VISIT', 'DONOR.IC_DATE_2PI_SIN', 'DONOR.IC_DATE_2PI_COS']]))
        else:
            config['design'] = '~ {factor} * SAMPLE.VISIT + {covariates}'.format(
                factor=safe_R_name(factor), covariates=' + '.join([c for c in config['columns'] if c not in ['SAMPLE.DONOR', 'DONOR.IC_TIME_REAL', 'SAMPLE.VISIT', safe_R_name(factor)]]))
        config['contrasts'] = _contrasts
        config['enr_contrasts'] = _enr_contrasts
        assert is_iterable(config['contrasts'][0])
        assert is_iterable(config['enr_contrasts'][0])

        # e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
        config['splines'] = None 
        config['splines_df'] = None
        
        if factor == 'season':
            config['drop_cols'] = ['DONOR.IC_DATE_2PI_SIN', 'DONOR.IC_DATE_2PI_COS']
        elif factor.startswith('DONOR:IC_'):
            continue
            config['drop_cols'] = ['{}{}'.format(safe_R_name(factor),
                                                 'True' if is_bool_dtype(_annot_df[factor].dtype) else 'M' if factor == 'DONOR:SEX' else '')]
        else:
            continue

        print(config['model'])
        
        config['block_on_donor'] = True
        config['remove_wrong_batch'] = True
        config['remove_exclusions'] = True
        config['remove_300BCG315'] = False
        config['remove_evening'] = True
        config['complete_design'] = False
        config['useful_samples'] = False
        config['save_non_log_CPM'] = False
        config['do_not_correct'] = None

        config['volcano_fdr'] = [0.05, 0.1, 0.2]

        config['effect_size_filter'] = None
        config['rank_metric'] = RANK_METRIC
        config['top_n'] = TOP_N

        config['lola_db_dir'] = LOLA_DB_DIR
        config['lola_db'] = LOLA_DB
        config['lola_collections'] = LOLA_COLLECTIONS
        config['lola_remove_replicates'] = True
        config['lola_metric'] = 'pValueLog'
        config['lola_show_n'] = 20
        config['lola_fdr'] = 0.05

        _pathways = FULL_BCG_PTHW_SET
        _regions = WHICH_REGIONS
        config['enrichr_gene_sets'] = _pathways
        config['enrichr_regions'] = _regions
        config['enrichr_metric'] = 'P-value'
        config['enrichr_show_n'] = 20
        config['enrichr_fdr'] = 0.1
        config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

        de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
        for d in ['LOLA', 'Enrichr']:
            make_dir(de_dir, d)
        with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False)

../metadata/complete_metadata.corrected.csv
fixed.donor_as_mixed.batch_corrected.sex.age.blood.TSS_enr.visit_time.season_interaction


#### V3 cytokine fold changes

In [9]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
_annot_df = get_sample_annot(fn=annot_fn)
print(annot_fn)

for visit in ['V3', 'V2']:
    for cyto in _annot_df.columns[_annot_df.columns.str.contains('^LFC_{visit}_CORR_CYTO:.*_good'.format(visit=visit))]:
        config = {}
        config['results_dir'] = RESULTS_DIR
        config['visits'] = ['V1']
        config['debug'] = False
        config['batch_corrected'] = False
        config['dream'] = False
        config['celltype'] = celltype
        config['model'] = 'V1{batch}{covar}{blood}.TSS_enr.visit_time{cyto}'.format(
            batch='.batch' if not config['batch_corrected'] else '.batch_corrected',
            covar='.sex.age{}'.format('.bmi.oralContra' if '_CORR_CM:' in cyto else ''),
            blood='.blood' if config['celltype'] == 'PBMC' else '',
            cyto='.{}'.format(safe_R_name(cyto))
        )

        config['annot_fn'] = annot_fn
        config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
        if config['batch_corrected']:
            config['counts_fn'] = os.path.join(DATA_DIR, 'batch_corrected_normalized_log2_CPM_{celltype}.csv.gz'.format(celltype=config['celltype']))
        else:
            config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))

        _contrasts = [[safe_R_name(cyto)]]
        _enr_contrasts = _contrasts
        _blood = safe_R_name(BLOOD)

        config['columns'] = sorted(set([safe_R_name(cyto), 'DONOR.SEX', 'DONOR.AGE', 'RUN.TSS_ENRICHMENT', 'SAMPLE.VISIT_TIME_REAL'] + \
        (['LAB.BATCH'] if not config['batch_corrected'] else []) + \
        (_blood if config['celltype'] == 'PBMC' else []) + \
        (['DONOR.BMI', 'DONOR.oralContraceptivesIncludingMen'] if '_CORR_CM:' in cyto else [])))

        config['design'] = '~ {covariates}'.format(covariates=' + '.join(config['columns']))

        config['contrasts'] = _contrasts
        config['enr_contrasts'] = _enr_contrasts
        assert is_iterable(config['contrasts'][0])
        assert is_iterable(config['enr_contrasts'][0])

        # e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
        config['splines'] = None 
        config['splines_df'] = None

        config['block_on_donor'] = False
        config['remove_wrong_batch'] = True
        config['remove_exclusions'] = True
        config['remove_300BCG315'] = False
        config['remove_evening'] = False
        config['complete_design'] = False
        config['useful_samples'] = False
        config['save_non_log_CPM'] = False
        config['do_not_correct'] = [safe_R_name(cyto)]

        config['volcano_fdr'] = [0.05, 0.1, 0.2]

        config['effect_size_filter'] = None
        config['rank_metric'] = RANK_METRIC
        config['top_n'] = TOP_N

        config['lola_db_dir'] = LOLA_DB_DIR
        config['lola_db'] = LOLA_DB
        config['lola_collections'] = LOLA_COLLECTIONS
        config['lola_remove_replicates'] = True
        config['lola_metric'] = 'pValueLog'
        config['lola_show_n'] = 20
        config['lola_fdr'] = 0.05

        _pathways = FULL_BCG_PTHW_SET
        _regions = WHICH_REGIONS
        config['enrichr_gene_sets'] = _pathways
        config['enrichr_regions'] = _regions
        config['enrichr_metric'] = 'P-value'
        config['enrichr_show_n'] = 20
        config['enrichr_fdr'] = 0.1
        config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

        de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
        for d in ['LOLA', 'Enrichr']:
            make_dir(de_dir, d)
        with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False)
        print(config['model'])

../metadata/complete_metadata.corrected.csv
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_GMCSF_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IL.10_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IL.1b_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IL.1ra_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IL.2R_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IL.6_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_IP10_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_MIG_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_MIP1a_good
V1.batch.sex.age.blood.TSS_enr.visit_time.LFC_V2_CORR_CYTO.C.albicans.yeast_24h_PBMC_MIP1b_good

#### Vaccination PBMC

In [10]:
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
print(annot_fn)
for suffix in ['_FC1.2_responder']:
    for donor_score_col in [
        'thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3',
        'thm.adaptive_MTB_7d_V3',
        'thm.heterologous_nonspecific_7d_V3',
    ]:
        _annot_df = get_sample_annot(annot_fn)
        assert (donor_score_col if not suffix else '{}{}'.format(donor_score_col, suffix)) in _annot_df.columns, donor_score_col

        config = {}
        config['results_dir'] = RESULTS_DIR
        config['debug'] = False
        config['batch_corrected'] = False
        config['dream'] = False

        config['celltype'] = celltype
        config['model'] = '{approach}.batch.sex.age{blood}.TSS_enr.visit_time.{score}{suffix}'.format(
            approach='dream' if config['dream'] else 'donor_as_mixed',\
            blood='.blood' if config['celltype'] == 'PBMC' else '',
            score=donor_score_col,
            suffix=suffix)

        config['annot_fn'] = annot_fn
        config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
        config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))

        if not suffix:
            _score = safe_R_name(donor_score_col)
            _interaction = '{score}*SAMPLE.VISIT'.format(score=_score)
            _contrasts = [[_score, 'V2', 'V3', '{}.V2'.format(_score), '{}.V3'.format(_score)]]
            _enr_contrasts = _contrasts
        else:
            _score = '{}{}'.format(safe_R_name(donor_score_col), safe_R_name(suffix))
            _interaction = '{score} + {score}:SAMPLE.VISIT'.format(score=_score)
            if config['dream']:
                _contrasts = [['{}{}R'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                               '{}{}N:SAMPLE.VISITV2'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                               '{}{}N:SAMPLE.VISITV3'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                               '{}{}R:SAMPLE.VISITV2'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                               '{}{}R:SAMPLE.VISITV3'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                              ]]
            else:
                _contrasts = [['{}{}R'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                               '{}{}N.V2'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                               '{}{}N.V3'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                               '{}{}R.V2'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                               '{}{}R.V3'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                              ]]
            _enr_contrasts = _contrasts

        _blood = safe_R_name(BLOOD)
        config['columns'] = ['SAMPLE.DONOR', 'SAMPLE.VISIT', 'DONOR.SEX', 'DONOR.AGE',
                             'RUN.TSS_ENRICHMENT', 'SAMPLE.VISIT_TIME_REAL', 'LAB.BATCH'] + \
        (_blood if config['celltype'] == 'PBMC' else []) + \
        [_score]

        config['design'] = '~ {interaction} + {covariates}'.format(
            interaction=_interaction,
            covariates=' + '.join(
                [c for c in config['columns'] if c not in [_score, 'SAMPLE.VISIT', 'SAMPLE.DONOR']]))

        config['contrasts'] = _contrasts
        config['enr_contrasts'] = [[c for c in _contrasts[0] if c.endswith('_R') or c.endswith('_R.V2') or c.endswith('_R.V3')]]
        assert is_iterable(config['contrasts'][0])
        assert is_iterable(config['enr_contrasts'][0])

        # e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
        config['splines'] = None 
        config['splines_df'] = None

        config['block_on_donor'] = True
        config['remove_wrong_batch'] = True
        config['remove_exclusions'] = True
        config['remove_300BCG315'] = False
        config['remove_evening'] = False
        config['complete_design'] = False
        config['useful_samples'] = False
        config['save_non_log_CPM'] = False
        config['do_not_correct'] = ['V2', 'V3']

        config['volcano_fdr'] = [0.05, 0.1, 0.2]

        config['effect_size_filter'] = None
        config['rank_metric'] = RANK_METRIC
        config['top_n'] = TOP_N

        config['lola_db_dir'] = LOLA_DB_DIR
        config['lola_db'] = LOLA_DB
        config['lola_collections'] = LOLA_COLLECTIONS
        config['lola_remove_replicates'] = True
        config['lola_metric'] = 'pValueLog'
        config['lola_show_n'] = 20
        config['lola_fdr'] = 0.05

        _pathways = FULL_BCG_PTHW_SET
        _regions = WHICH_REGIONS
        config['enrichr_gene_sets'] = _pathways
        config['enrichr_regions'] = _regions
        config['enrichr_metric'] = 'P-value'
        config['enrichr_show_n'] = 20
        config['enrichr_fdr'] = 0.1
        config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

        de_dir = make_dir(RESULTS_DIR, 'DE', '{}.{}'.format(config['celltype'], config['model']))
        for d in ['LOLA', 'Enrichr']:
            make_dir(de_dir, d)
        with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False)
        print(config['model'])

../metadata/complete_metadata.corrected.csv
donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.adaptive_MTB_7d_V3_FC1.2_responder
donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.heterologous_nonspecific_7d_V3_FC1.2_responder


### Bootstraps

In [5]:
DEBUG = False
celltype = 'PBMC'
annot_fn = SAMPLE_ANNOT_FN.format(which='.corrected')
print(annot_fn)
for suffix in ['_FC1.2_responder']:
    for donor_score_col in [
        'thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3',
        # 'thm.adaptive_MTB_7d_V3',
        # 'thm.heterologous_nonspecific_7d_V3',
    ]:
        for seed in range(101, 1001):
            _annot_df = get_sample_annot(annot_fn)
            assert (donor_score_col if not suffix else '{}{}'.format(donor_score_col, suffix)) in _annot_df.columns, donor_score_col

            config = {}
            config['results_dir'] = RESULTS_DIR
            config['bootstrap'] = seed
            config['debug'] = DEBUG
            config['batch_corrected'] = False
            config['dream'] = False

            config['celltype'] = celltype
            config['model'] = 'BS{bootstrap}.{approach}.batch.sex.age{blood}.TSS_enr.visit_time.{score}{suffix}'.format(
                bootstrap=seed,
                approach='dream' if config['dream'] else 'donor_as_mixed',\
                blood='.blood' if config['celltype'] == 'PBMC' else '',
                score=donor_score_col,
                suffix=suffix)

            config['annot_fn'] = annot_fn
            config['peak_fn'] = os.path.join(DATA_DIR, 'peaks_filtered_PBMC.csv.gz')
            config['counts_fn'] = os.path.join(DATA_DIR, 'quantification_filtered_{celltype}.csv.gz'.format(celltype=config['celltype']))

            if not suffix:
                _score = safe_R_name(donor_score_col)
                _interaction = '{score}*SAMPLE.VISIT'.format(score=_score)
                _contrasts = [[_score, 'V2', 'V3', '{}.V2'.format(_score), '{}.V3'.format(_score)]]
                _enr_contrasts = _contrasts
            else:
                _score = '{}{}'.format(safe_R_name(donor_score_col), safe_R_name(suffix))
                _interaction = '{score} + {score}:SAMPLE.VISIT'.format(score=_score)
                if config['dream']:
                    _contrasts = [['{}{}R'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                                   '{}{}N:SAMPLE.VISITV2'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                                   '{}{}N:SAMPLE.VISITV3'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                                   '{}{}R:SAMPLE.VISITV2'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                                   '{}{}R:SAMPLE.VISITV3'.format(safe_R_name(donor_score_col), safe_R_name(suffix)),
                                  ]]
                else:
                    _contrasts = [['{}{}R'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                                   '{}{}N.V2'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                                   '{}{}N.V3'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                                   '{}{}R.V2'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                                   '{}{}R.V3'.format(safe_R_name(donor_score_col), safe_R_name(suffix.replace('_responder', '_'))),
                                  ]]
                _enr_contrasts = _contrasts

            _blood = safe_R_name(BLOOD)
            config['columns'] = ['SAMPLE.DONOR', 'SAMPLE.VISIT', 'DONOR.SEX', 'DONOR.AGE',
                                 'RUN.TSS_ENRICHMENT', 'SAMPLE.VISIT_TIME_REAL', 'LAB.BATCH'] + \
            (_blood if config['celltype'] == 'PBMC' else []) + \
            [_score]

            config['design'] = '~ {interaction} + {covariates}'.format(
                interaction=_interaction,
                covariates=' + '.join(
                    [c for c in config['columns'] if c not in [_score, 'SAMPLE.VISIT', 'SAMPLE.DONOR']]))

            config['contrasts'] = _contrasts
            config['enr_contrasts'] = [[c for c in _contrasts[0] if c.endswith('_R') or c.endswith('_R.V2') or c.endswith('_R.V3')]]
            assert is_iterable(config['contrasts'][0])
            assert is_iterable(config['enr_contrasts'][0])

            # e.g., "DONOR.IC_DATE_REAL" # this will create "SPLINES" variables - use it in the design
            config['splines'] = None 
            config['splines_df'] = None

            config['block_on_donor'] = True
            config['remove_wrong_batch'] = True
            config['remove_exclusions'] = True
            config['remove_300BCG315'] = False
            config['remove_evening'] = False
            config['complete_design'] = False
            config['useful_samples'] = False
            config['save_non_log_CPM'] = False
            config['do_not_correct'] = ['V2', 'V3']

            config['volcano_fdr'] = [0.05, 0.1, 0.2]

            config['effect_size_filter'] = None
            config['rank_metric'] = RANK_METRIC
            config['top_n'] = TOP_N

            config['lola_db_dir'] = LOLA_DB_DIR
            config['lola_db'] = LOLA_DB
            config['lola_collections'] = LOLA_COLLECTIONS
            config['lola_remove_replicates'] = True
            config['lola_metric'] = 'pValueLog'
            config['lola_show_n'] = 20
            config['lola_fdr'] = 0.05

            _pathways = FULL_BCG_PTHW_SET
            _regions = WHICH_REGIONS
            config['enrichr_gene_sets'] = _pathways
            config['enrichr_regions'] = _regions
            config['enrichr_metric'] = 'P-value'
            config['enrichr_show_n'] = 20
            config['enrichr_fdr'] = 0.1
            config['enrichr_db_dir'] = os.path.join(METADATA, 'gene_set_libraries') 

            de_dir = make_dir(RESULTS_DIR, 'DE', f'{config["celltype"]}.{config["model"]}')
            for d in ['LOLA', 'Enrichr']:
                make_dir(de_dir, d)
            with open(de_fn(config['celltype'], config['model'], data='config', ext='yml', gzipped=False, results_dir=RESULTS_DIR), 'w') as f:
                yaml.dump(config, f, default_flow_style=False, sort_keys=False)
            print(config['model'])

../metadata/complete_metadata.corrected.csv
BS101.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS102.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS103.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS104.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS105.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS106.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS107.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_wo_LAC_IL10_IL1ra_V3_FC1.2_responder
BS108.donor_as_mixed.batch.sex.age.blood.TSS_enr.visit_time.thm.innate_nonspecific_24h_w